<a href="https://colab.research.google.com/github/ArfaKhalid/Spatial-Computing/blob/main/GGS590_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Task # 1
**a.** Using an object oriented approach, for the capital of Togo (Lomé) write code to estimate the geographic land area within the GID 2 region utilizing GADM boundaries. Look for the Lomé region polygon, which may be either “TGO.3.1_1” or “TGO.3.6_2” depending on your GADM data version. Make sure you use a projected Coordinate Reference System. Report your estimate to the closest square kilometer.

In [6]:
# Loading data - GADM
from google.colab import files
uploaded = files.upload()

Saving gadm41_TGO_2.prj to gadm41_TGO_2.prj
Saving gadm41_TGO_2.shp to gadm41_TGO_2.shp
Saving gadm41_TGO_2.shx to gadm41_TGO_2.shx
Saving gadm41_TGO_2.cpg to gadm41_TGO_2.cpg
Saving gadm41_TGO_2.dbf to gadm41_TGO_2.dbf


In [59]:
# Loading data - Building
from google.colab import files
uploaded = files.upload()

Saving subset_buildings_togo.cpg to subset_buildings_togo.cpg
Saving subset_buildings_togo.dbf to subset_buildings_togo.dbf
Saving subset_buildings_togo.prj to subset_buildings_togo.prj
Saving subset_buildings_togo.qmd to subset_buildings_togo.qmd
Saving subset_buildings_togo.shp to subset_buildings_togo.shp
Saving subset_buildings_togo.shx to subset_buildings_togo.shx


In [60]:
# Loading data - Railways
from google.colab import files
uploaded = files.upload()

Saving gis_osm_railways_free_1.cpg to gis_osm_railways_free_1.cpg
Saving gis_osm_railways_free_1.dbf to gis_osm_railways_free_1.dbf
Saving gis_osm_railways_free_1.prj to gis_osm_railways_free_1.prj
Saving gis_osm_railways_free_1.shp to gis_osm_railways_free_1.shp
Saving gis_osm_railways_free_1.shx to gis_osm_railways_free_1.shx


In [62]:
import geopandas as gpd
# Load GADM boundaries for Togo
class Region:
    def __init__(self, gadm_file_path, region_code):
        self.gadm_file_path = gadm_file_path
        self.region_code = region_code
        self.region_gdf = self.load_region()

    def load_region(self):
        gadm_gdf = gpd.read_file(self.gadm_file_path)
        # Filter the GeoDataFrame for the specified Lomé region
        lome_gdf = gadm_gdf[gadm_gdf['GID_2'] == self.region_code]
        if lome_gdf.empty:
            raise ValueError(f"No region found with GID_2 = {self.region_code}")

        # Project the GeoDataFrame to a projected CRS
        lome_gdf = lome_gdf.to_crs("EPSG:3857")
        return lome_gdf

    def get_area_km2(self):
        # Calculate the total area of the geometry in square kilometers
        return self.region_gdf.geometry.area.sum() / 1e6


In [63]:
# AreaCalculator Class
class AreaCalculator:
    def __init__(self, gadm_path, region_code):
        self.region = Region(gadm_path, region_code)

    def calculate_region_area(self):
        # Run method to get the area from the Region class
        return self.region.get_area_km2()

    def report_area(self):
        # Print the area to the nearest square kilometer
        area_km2 = self.calculate_region_area()
        print(f"Total area of Lomé region (GID_2 = {self.region.region_code}): {round(area_km2)} km²")


In [64]:
# set the path to GADM shapefile
gdam_path = "gadm41_TGO_2.shp"
# set the region code for Lomé
region_code = "TGO.3.6_2"
# Calculate area
calculator = AreaCalculator(gdam_path, region_code)
calculator.report_area()

Total area of Lomé region (GID_2 = TGO.3.6_2): 97 km²


### Task # 1
**b.**Using an object oriented approach, estimate the geographic area taken up by buildings within the
same Lomé region, using OpenStreetMap data. You will need to undertake an intersection,
followed by finding the area on your remaining GeoPandas polygons. Think critically about this
process, especially about the size of the data you are processing. If needed, there is a subset of
building data on the MyMason course content page.

In [65]:
import geopandas as gpd

class Region:
    def __init__(self, gadm_file_path, region_code):
        self.gadm_file_path = gadm_file_path
        self.region_code = region_code
        self.region_gdf = self.load_region()

    def load_region(self):
        # Load GADM shapefile
        gadm_gdf = gpd.read_file(self.gadm_file_path)
        # Filter to get only the Lomé region
        region_gdf = gadm_gdf[gadm_gdf['GID_2'] == self.region_code]
        # For a projected CRS for area calculations
        region_gdf = region_gdf.to_crs("EPSG:3857")
        return region_gdf


In [67]:
# Building area calculator class
class BuildingAreaCalculator:
    def __init__(self, osm_buildings_file_path):
        self.osm_buildings_file_path = osm_buildings_file_path
        self.buildings_gdf = self.load_building_data()

    def load_building_data(self):
        # Load OSM building data
        buildings_gdf = gpd.read_file(self.osm_buildings_file_path)
        # Set to a projected CRS for area calculations
        buildings_gdf = buildings_gdf.to_crs("EPSG:3857")
        return buildings_gdf

    def calculate_building_area(self, region):
        # Intersect buildings with the Lomé region boundary
        intersected_gdf = gpd.overlay(self.buildings_gdf, region.region_gdf, how="intersection")
        # Calculate area in square kilometers
        building_area_km2 = intersected_gdf.geometry.area.sum() / 1e6
        return building_area_km2


In [68]:
# File paths and region code
gadm_path = "gadm41_TGO_2.shp"
osm_buildings_path = "subset_buildings_togo.shp"
region_code = "TGO.3.6_2"

# Initialize classes
region = Region(gadm_path, region_code)
building_calculator = BuildingAreaCalculator(osm_buildings_path)

# Calculate building area within Lomé
building_area_km2 = building_calculator.calculate_building_area(region)
print(f"Total building area within Lomé region: {building_area_km2:.2f} km²")


Total building area within Lomé region: 0.73 km²


### Task 1
**c.**  Using an object oriented approach, estimate the length of railroad within the Lomé region, using
OpenStreetMap data. You will need to undertake an intersection, followed by finding the length of
your remaining GeoPandas LineStrings.

In [69]:
import geopandas as gpd

class Region:
    def __init__(self, gadm_file_path, region_code):
        self.gadm_file_path = gadm_file_path
        self.region_code = region_code
        self.region_gdf = self.load_region()

    def load_region(self):
        gadm_gdf = gpd.read_file(self.gadm_file_path)
        # Filter for the Lomé region based on the region code
        lome_gdf = gadm_gdf[gadm_gdf['GID_2'] == self.region_code]
        # Ensure using a projected CRS for length calculations
        lome_gdf = lome_gdf.to_crs("EPSG:3857")
        return lome_gdf


In [70]:
class OSMData:
    def __init__(self, osm_file_path, data_type="railroad"):
        self.osm_file_path = osm_file_path
        self.data_type = data_type
        self.data_gdf = self.load_data()

    def load_data(self):
        osm_gdf = gpd.read_file(self.osm_file_path)
        # Ensure CRS matches the Lomé region CRS
        osm_gdf = osm_gdf.to_crs("EPSG:3857")
        return osm_gdf

    def calculate_railroad_length_within_region(self, region):
        # Intersect railroad data with region boundary
        intersected_gdf = gpd.overlay(self.data_gdf, region.region_gdf, how="intersection")
        # Calculate length in kilometers
        railroad_length_km = intersected_gdf.geometry.length.sum() / 1e3
        return railroad_length_km



In [71]:
class LomeRailroadCalculator:
    def __init__(self, gadm_path, region_code, osm_railroad_path):
        self.region = Region(gadm_path, region_code)
        self.railroad_data = OSMData(osm_railroad_path, "railroad")

    def calculate_railroad_length(self):
        return self.railroad_data.calculate_railroad_length_within_region(self.region)

    def report_results(self):
        print(f"Total railroad length within Lomé region: {self.calculate_railroad_length():.2f} km")


In [72]:
# Paths and region code for Lomé
gadm_path = "gadm41_TGO_2.shp"
osm_railroad_path = "gis_osm_railways_free_1.shp"
region_code = "TGO.3.6_2"

# Calculations
calculator = LomeRailroadCalculator(gadm_path, region_code, osm_railroad_path)
calculator.report_results()


Total railroad length within Lomé region: 32.38 km
